In [1]:
# Install Dependencies
# %pip install -qU datasets pinecone-client sentence-transformers torch pyngrok

# Citations
### Retriever Model
The retriever model converts text snippets into vectors and vice versa. The model used is a pretrained
  -  @inproceedings{reimers-2019-sentence-bert,
    title = "Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks",
    author = "Reimers, Nils and Gurevych, Iryna",
    booktitle = "Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing",
    month = "11",
    year = "2019",
    publisher = "Association for Computational Linguistics",
    url = "https://arxiv.org/abs/1908.10084",
    }
### Generator Model
This model is a generative ai that generates text upon input. The model used is BART pretrained for Q&A
- https://yjernite.github.io/lfqa.html
### Tutorial
As we are complete beginners, we heavily utilized the youtube video "Open Source Generative AI in Question-Answering (NLP) using Python" by James Brigg for the AI part
- https://invidious.private.coffee/watch?v=L8U-pm-vZ4c




In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration
from sentence_transformers import SentenceTransformer
import torch
import pinecone
from config import PINECONE_API_KEY

#Connect to vector DB
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment="gcp-starter"
)
#basically a table in a db except it's not sql, hardcoded here because
# free plan only lets you make one anyways
index = pinecone.Index('genghis-khan')

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'


# load the retriever model from huggingface model hub
retriever = SentenceTransformer(
    "flax-sentence-embeddings/all_datasets_v3_mpnet-base", device=device)
retriever


# load generator model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained(
    'vblagoje/bart_lfqa').to(device)


In [3]:
from pprint import pprint
# Helper functions for retrieval and answering

# returns the k vectors that most match the query in vector db
def query_pinecone(query, top_k):
    # generate embeddings for the query
    query = retriever.encode([query]).tolist()
    # return relevant context vecros from query
    context = index.query(query, top_k=top_k, include_metadata=True)
    return context


# this function checks if relevant passages for the query exists
# pinecone gives a "score" metric which shows vector similarity 
def check_relevancy(query):
    query = retriever.encode([query]).tolist()
    context = index.query(query, top_k=1, include_metadata=True)

    # Access the score value
    score = context['matches'][0]['score']

    # Check if the score is greater than 0.5
    # this is an arbitrary value, but I've seen that
    # a high match is around 0.8 while low is around 0.3
    return score


def generate_answer(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    context = [f"<P> {m['metadata']['passage_text']}" for m in context]
    # concatinate all context passages
    context = " ".join(context)

    # contcatinate the query and context passages
    input_text = f"question: {query} context: {context}"

    # tokenize the query to get input_ids
    input_tokens = tokenizer([input_text], max_length=1024,
                       return_tensors="pt").to(device)

    # use generator to predict output ids
    ids = generator.generate(
        input_tokens["input_ids"], num_beams=2, min_length=20, max_length=100)
        
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(
        ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return answer


In [4]:
# Import modules
from flask import Flask, request, jsonify, make_response
from werkzeug.serving import make_server
from pyngrok import ngrok
import threading

# Create App
app = Flask(__name__)


def run_app():
    global server
    server = make_server('127.0.0.1', 5000, app)
    server.serve_forever()



In [5]:
# Define Routes
from tqdm import tqdm
import pandas as pd
import pinecone


@app.route("/")
def home():
    return "Api works!"


@app.route("/test_route")
def test_route():
    return jsonify("test")

@app.route('/ask', methods=['POST'])
def ask():
    try:
        if request.method == 'POST':
            # Get the search query from the form
            query = request.form['search_query']

            context = query_pinecone(query, top_k=3)
            #debug statement to view context and scores
            print(context)

            relevancy_of_text = check_relevancy(query)
            if relevancy_of_text>0.5:
                matching_context = context["matches"]
                answer = generate_answer(query, matching_context)
                citations = [doc["metadata"]['passage_text']
                             for doc in matching_context]
                return jsonify({"answer": answer, "citations": citations, "confidence":relevancy_of_text})
            else:
                return jsonify({"message": "No relevant text found"})

    except Exception as e:
        print("An error occurred: ", e)
        return jsonify({'error': 'Internal Server Error'}), 500

@app.route('/vectorize_page', methods=['POST'])

# performance: the genghis khan wikipedia page takes around 40 secs
def vectorize_page():
    try:
        # Get the JSON data from the request body
        data = request.json
        print("data: " + str(data))

        # Check if 'url' key exists in the JSON data
        if 'name' not in data:
            return jsonify({'error': 'The JSON data must contain a "name" key'}), 400

        index_name = data['name']

        if index_name not in pinecone.list_indexes():
            pinecone.create_index(
                index_name,
                dimension=768,
                metric="cosine"
            )

        index = pinecone.Index(index_name)

        if 'passage_text' not in data:
            return jsonify({'error': 'No passages found'}), 400

        passage_texts = data['passage_text']
        df = pd.DataFrame({'passage_text': passage_texts})

        # Drop rows with less than 2 words
        df = df[df['passage_text'].apply(lambda x: len(x.split()) >= 2)]

        batch_size = len(df)
        emb = retriever.encode(df["passage_text"].tolist()).tolist()
        meta = df.to_dict(orient="records")
        ids = [f"{idx}" for idx in range(len(df))]
        to_upsert = list(zip(ids, emb, meta))

        index = pinecone.Index(index_name)

        for i in tqdm(range(len(to_upsert))):
            _ = index.upsert(vectors=[to_upsert[i]])

        index_stats = index.describe_index_stats()
        return jsonify({'message': f'{index_name} vectorized'})

    except Exception as e:
        print("An error occurred: ", e)
        return jsonify({'error': 'Internal Server Error'}), 500


In [6]:
# launch server
flask_thread = threading.Thread(target=run_app)
flask_thread.start()

# Open an ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000)
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:5000\"".format(public_url))


t=2023-10-01T06:26:43-0400 lvl=warn msg="ngrok config file found at legacy location, move to XDG location" xdg_path="/Users/notAdmin/Library/Application Support/ngrok/ngrok.yml" legacy_path=/Users/notAdmin/.ngrok2/ngrok.yml


 * ngrok tunnel "NgrokTunnel: "https://ccef-37-120-244-62.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"


In [7]:
# # Shutdown Api
# if server:
#     server.shutdown()
# # Disconnect the ngrok tunnel
# ngrok.disconnect(public_url)


{'matches': [{'id': '125',
              'metadata': {'passage_text': 'The tribes of the Mongol steppe '
                                           'had no fixed succession system, '
                                           'but often defaulted to some form '
                                           'of ultimogeniture—succession of '
                                           'the youngest son—on the grounds '
                                           'that unlike his older brothers, '
                                           'the youngest son would not have '
                                           'had time to gain a following for '
                                           'himself and needed the help of his '
                                           "father's inheritance.[163] "
                                           'However, it has been noticed that '
                                           'this inheritance applied only to '
                                  

127.0.0.1 - - [01/Oct/2023 06:27:32] "POST /ask HTTP/1.1" 200 -
